In [1]:
import numpy as np
from scipy import signal
from scipy.interpolate import interp1d
from scipy.signal import butter, filtfilt, iirdesign, zpk2tf, freqz, square
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn
# set the seed
np.random.seed(2)

## Match filter on regular sampling:
* Since we have a regular sampling there is no problem with the frequencies of the dictionary or the nyquist frequency, this allow us to use an easy procedure for the match filter method for the fourier dictionary and with this, we will use the fast fourier algorithm.

## Cases
* We are going to consider different cases in order to see how this method will work, this cases are:
    * Signal only of white noise
    * Signal of a sinusoid plus white noise
    * Signal of a composition of sinusoids plus white noise

## Creating the signal

In [2]:
class Signal:
    def __init__(self, frequencies=[1], weights=[1], noise=None):
        """
        it will generate a signal usign the frequencies and the weight from input.
        the frecuencies should be in Hertz and the weight is a constante of "amplitud" 
        of every sub-signal associated to avery frequency.
        weight should be a number between 0 and 1
        """
        self.frequencies = frequencies
        self.weights = weights
        self.noise = noise
        self.amp = 1
    
    def noise_samples(self):
        return self.noise
    
    def sin_samples(self, times, with_noise=False):
        y = np.zeros(times.shape[-1])
        for i in range(len(self.frequencies)):
            y += self.weights[i] * np.sin(2 * np.pi * self.frequencies[i] * times)
        if with_noise:
            y += self.noise
        return self.amp * y
    
    def square_samples(self, times, with_noise=False):
        y = np.zeros(len(times))
        for i in range(len(self.frequencies)):
            y += self.weights[i] * np.sign(np.sin(2 * np.pi * self.frequencies[i] * times))
        if with_noise:
            y += self.noise
        return self.amp * y
    
    def gaussian_samples(self):
        pass

## The Match Filter method:
* The idea is to make the convolution between the signal and a template, using the fourier transform allow us to make easy the convolution. The procedure is:
    * First make the fourier transform of the signal and template
    * do the product and scale for the power spectral density of the noise
    * then go back to the time domain in order to se how is the SNR for different matches across time.
    * find the best SNR, i.e. the SNR when the template and the signal match the best while we are "moving" the template across time.

In [4]:
# Match_filter
class MatchFilter:
    def __init__(self, signal, template, dt):
        self.dt = dt
        self.fs = 0.5 / dt
        self.signal = signal
        self.template = template
        self.freq = self.compute_freqs()
        self.psd = self.make_psd(signal)
        
    def compute_freqs(self):
        return np.fft.fftfreq(len(self.signal), self.dt)
    
    def make_psd(self, data):
        power, freq = mlab.psd(data, Fs=self.fs, NFFT=4 * self.fs)
        return np.interp(self.freqs, freq, power)
    
    def product(self, signal_fft, template_fft, power_vec):
        return signal_fft * template_fft.conjugate() / power_vec 
        
    def SNR(self, noise, normalize=True): # as defined from LIGO
        template_fft = np.fft.fft(self.template)
        signal_fft = np.fft.fft(self.signal)
        power_vec = self.make_psd(noise)
        optimal = 2* np.fft.ifft(self.product(signal_fft, template_fft, power_vec)) * self.fs
        
        # in order to get an SNR of 1 when the template is at a time of where the signal is only noise,
        # we divide by this sigma:
        if normalize:
            df = np.abs(self.freq[1] - self.freq[2])
            sigmasq = 1*(template_fft * template_fft.conjugate() / power_vec).sum() * df
            sigma = np.sqrt(np.abs(sigmasq))
        else:
            sigma = 1
            
        SNR_complex = optimal / sigma
        
        # this wil give us an array of matches between the signal and the template representing
        # the offset in time, this mean that we will get the maximun SNR at offset 0 and the same value at the end
        # of the array (maximun offset), this is because is cyclical, so in order to get the maximun SNR at an
        # offset of time between the minimum (offset 0) and maximum, we move the template peak to the end and sinse we usually
        # have the peak of the template at the middle of time, we do:
        
        peaksample = int(self.template.size / 2)  # location of peak in the template
        SNR_complex = np.roll(SNR_complex,peaksample) # we shift the SNR because we want to shift the time of the template.
        return SNR_complex

### Testing for White noise
* No matter what template whe try here there should be always a bad SNR. This is a trivial test